# 01. The Target Model

Our target model is available in HuggingFace: [TsinghuaC3I/Llama-3.1-8B-UltraMedical](https://huggingface.co/TsinghuaC3I/Llama-3.1-8B-UltraMedical).

`Llama-3.1-8B-UltraMedical` is an open-access large language model (LLM) specialized in biomedicine. Developed by the Tsinghua C3I Lab, this model aims to enhance medical examination access, literature comprehension, and clinical knowledge.

Building on the foundation of Meta's `Llama-3.1-8B`, `Llama-3.1-8B-UltraMedical` is trained on our UltraMedical collection with supervised fine-tuning (SFT), iterative preference learning (like DPO and KTO). The UltraMedical collection is a large-scale, high-quality dataset of biomedical instructions, comprising 410,000 synthetic and manually curated samples, along with more than 100,000 preference data.

In this notebook we will detail the following points:
1. Download the target model from Hugging Face in your local directory.
2. Configure the target model checkpoint.
3. Upload the target model on SambaStudio. 

## Outline
- [1. Setup](#1-setup)
    - [1.1 Imports](#11-imports)
    - [1.2 Instantiate the SambaStudio client for BYOC](#12-instantiate-the-sambastudio-client-for-byoc)
- [2. Target model: `OpenBioLLM-8B`](#2-target-model-openbiollm-8b)
    - [2.1 Download the target model from HuggingFace](#21-download-the-target-model-from-huggingface)
    - [2.2 Configure checkpoint](#22-configure-checkpoint)
    - [2.3 Set and check chat template (optional)](#23-set-and-check-chat-template-optional)
    - [2.4 Set padding token (required for training)](#24-set-padding-token-required-for-training)
    - [2.5 Get model params and Sambastudio suitable Apps](#25-get-model-params-and-sambastudio-suitable-apps)
- [3. Upload the checkpoint to SambaStudio](#3-upload-the-checkpoint-to-sambastudio)
    - [3.1 Using the checkpoint dictionary](#31-using-the-checkpoint-dictionary)
    - [3.2 Using the `target_checkpoint_config.yaml`](#32-using-the-target_checkpoint_configyaml)

## 1. Setup

In [1]:
# Import libraries
import os
import sys
import re
import json
import time
import yaml
from pprint import pprint
from huggingface_hub import hf_hub_download, HfApi

# Get absolute paths for kit_dir and repo_dir
current_dir = os.getcwd()
kit_dir =  os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(kit_dir, '..'))

# Adding directories to the Python module search path
sys.path.append(repo_dir)

# Bring Your Own Checkpoint (BYOC)
from utils.byoc.src.snsdk_byoc_wrapper import BYOC

/Users/francescar/Documents/ai-starter-kit/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Instantiate the BYOC (Bring Your Own Checkpoint) SambaStudio client
byoc = BYOC()

2025-04-30 11:49:17,046 [INFO] Using variables from .snapi config to set up Snsdk.


In [3]:
# Load the target model config
config_target_yaml = '01_config_target.yaml'

# Open and load the YAML file into a dictionary
with open(config_target_yaml, 'r') as file:
    config_target = yaml.safe_load(file)['model']

pprint(config_target)

{'description': 'Llama-3.1-8B-UltraMedical is an open-access large language '
                'model (LLM) specialized in biomedicine. Developed by the '
                'Tsinghua C3I Lab, this model aims to enhance medical '
                'examination access, literature comprehension, and clinical '
                'knowledge.',
 'hf_name': 'TsinghuaC3I/Llama-3.1-8B-UltraMedical',
 'model_name': 'Llama-3.1-8B-UltraMedical',
 'param_count': 8,
 'publisher': 'TsinghuaC3I'}


## 2. Target model: `OpenBioLLM-8B`

### 2.1 Download the target model from HuggingFace
You can use your own fine-tuned models or you can download and use [Huggingface model checkpoints](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending).

In our example we will use an available model in HuggingFace: [TsinghuaC3I/Llama-3.1-8B-UltraMedical](https://huggingface.co/TsinghuaC3I/Llama-3.1-8B-UltraMedical).

Llama-3.1-8B-UltraMedical is an open-access large language model (LLM) specialized in biomedicine. Developed by the Tsinghua C3I Lab, this model aims to enhance medical examination access, literature comprehension, and clinical knowledge.

Building on the foundation of Meta's Llama-3.1-8B, Llama-3.1-8B-UltraMedical is trained on our UltraMedical collection with supervised fine-tuning (SFT), iterative preference learning (like DPO and KTO). The UltraMedical collection is a large-scale, high-quality dataset of biomedical instructions, comprising 410,000 synthetic and manually curated samples, along with more than 100,000 preference data.

In [4]:
# Hugging Face model name
hf_model = config_target['hf_name']
# Our local target directory
target_dir = os.path.join(kit_dir, 'data', 'models') 

In [5]:
# Create target dir if it does not exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Download checkpoint to your target directory
repo_files = HfApi().list_repo_files(hf_model)
for file_name in repo_files:
    hf_hub_download(repo_id=hf_model, filename=file_name, cache_dir=target_dir)

In [6]:
# Find the snapshot folder inside your target directory
for root, dirs, files in os.walk(target_dir):
    if "snapshots" in root and hf_model.replace("/", "--") in root:
        checkpoint_folder = os.path.join(root, dirs[0])
        break

print("Checkpoint folder: ", checkpoint_folder)

Checkpoint folder:  /Users/francescar/Documents/ai-starter-kit/e2e_draft_model_training/data/models/models--TsinghuaC3I--Llama-3.1-8B-UltraMedical/snapshots/c982b9e34f7cf396f41627e7d851629f9e4e5792


### 2.2 Configure checkpoint

Some parameters should be provided as a checkpoint dictionary, in order to upload a previously created checkpoint.

In [7]:
# Initialise the checkpoint dictionary
checkpoint = {
    'model_name': config_target['model_name'],
    'publisher': config_target['publisher'],
    'description': config_target['description'],
    'param_count': config_target['param_count'],
    'checkpoint_path': checkpoint_folder
}

### 2.3 Set and check chat template (optional) 

If you want to use chat templates (roles structures), you need to include or update the existing chat template.

This should be formatted as a Jinja2 String template as in the following `llama` example.

In [8]:
# Jinjia chat template
jinja_chat_template = """ 
{% for message in messages %}
    {% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>' + '\n' + message['content'] | trim + '<|eot_id|>'+'\n' %}
    {% if loop.index0 == 0 %}{% set content = bos_token + content %}
    {% endif %}
    {{content}}
{% endfor %}
{{'<|start_header_id|>assistant<|end_header_id|>'+'\n'}}
"""
# Delete escape characters
jinja_chat_template = re.sub(r"(?<!')\n(?!')", "", jinja_chat_template).strip().replace('  ','')

In [9]:
# Open and update your tokenizer config from the checkpoint path
with open(os.path.join(checkpoint['checkpoint_path'], 'tokenizer_config.json'), 'r+') as file:
    data = json.load(file)
    data['chat_template'] = jinja_chat_template
    file.seek(0)
    file.truncate()
    json.dump(data, file, indent=4)

In [10]:
# Render template when using a roles / chat structure
test_messages = [
    {"role": "system", "content": "This is a system prompt."},
    {"role": "user", "content": "This is a user prompt."},
    {"role": "assistant", "content": "This is a response from the assistant."},
    {"role": "user", "content": "This is an user follow up"}
    ]
byoc.check_chat_templates(test_messages, checkpoint_paths=checkpoint['checkpoint_path'])

2025-04-30 11:49:20,293 [INFO] Raw chat template for checkpoint in /Users/francescar/Documents/ai-starter-kit/e2e_draft_model_training/data/models/models--TsinghuaC3I--Llama-3.1-8B-UltraMedical/snapshots/c982b9e34f7cf396f41627e7d851629f9e4e5792:
{% for message in messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>' + '
' + message['content'] | trim + '<|eot_id|>'+'
' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{content}}{% endfor %}{{'<|start_header_id|>assistant<|end_header_id|>'+'
'}}

2025-04-30 11:49:20,298 [INFO] Rendered template with input test messages:

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
This is a system prompt.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
This is a user prompt.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
This is a response from the assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
This is an user follow up<|eot_id|>
<|start_header_id|>

### 2.4 Set padding token (required for training)

If `pad_token_id` is not set in your checkpoint configuration yet and you want to do a further fine-tuning over your checkpoint, you need to set `pad_token_id` in your checkpoint `config.json`.

In [11]:
# Adding pad_token_id to checkpoint config
with open(os.path.join(checkpoint['checkpoint_path'], 'config.json'), 'r+') as file:
    data = json.load(file)
    data['pad_token_id'] = None
    file.seek(0)
    file.truncate()
    json.dump(data, file, indent=4)

### 2.5 Get model params and Sambastudio suitable apps

Extra parameters are required to upload your checkpoint, including model architecture, sequence length, and vocabulary size.

These parameters can be extracted from your checkpoint configuration, and included in your checkpoint dictionary parameters.


In [12]:
# Extract all the checkpoint parameters
checkpoint_config_params = byoc.find_config_params(checkpoint_paths=checkpoint['checkpoint_path'])[0]
# Update your checkpoint dictionary
checkpoint.update(checkpoint_config_params)

2025-04-30 11:49:20,306 [INFO] Params for checkpoint in /Users/francescar/Documents/ai-starter-kit/e2e_draft_model_training/data/models/models--TsinghuaC3I--Llama-3.1-8B-UltraMedical/snapshots/c982b9e34f7cf396f41627e7d851629f9e4e5792:
[{'model_arch': 'llama', 'seq_length': 131072, 'vocab_size': 128256}]


In order to upload a model checkpoint, you need to set a SambaStudio App.

You can search for suitable apps using the checkpoint parameters, and then select the best match.

In [13]:
# Look for suitable apps in SambaStudio
suitable_apps = byoc.get_suitable_apps(checkpoint)

2025-04-30 11:49:21,181 [INFO] Checkpoint Llama-3.1-8B-UltraMedical suitable apps:
[{'id': 'eb0aaad1-694f-41b6-958a-b974737635c4', 'name': 'Samba1 Llama3.1 Experts'}]


In [14]:
# From the three suitable apps found, we will use the last one as it is more generic
checkpoint["app_id"] = suitable_apps[-1][0]['id']

In [15]:
# We can see here all the parameters required to upload the checkpoint
print(checkpoint)

{'model_name': 'Llama-3.1-8B-UltraMedical', 'publisher': 'TsinghuaC3I', 'description': 'Llama-3.1-8B-UltraMedical is an open-access large language model (LLM) specialized in biomedicine. Developed by the Tsinghua C3I Lab, this model aims to enhance medical examination access, literature comprehension, and clinical knowledge.', 'param_count': 8, 'checkpoint_path': '/Users/francescar/Documents/ai-starter-kit/e2e_draft_model_training/data/models/models--TsinghuaC3I--Llama-3.1-8B-UltraMedical/snapshots/c982b9e34f7cf396f41627e7d851629f9e4e5792', 'model_arch': 'llama', 'seq_length': 131072, 'vocab_size': 128256, 'app_id': 'eb0aaad1-694f-41b6-958a-b974737635c4'}


## 3. Upload the checkpoint to SambaStudio

### 3.1 Using the checkpoint dictionary

In [16]:
# Call the `upload_checkpoint method` from client with your checkpoint parameters (this can take a while)
model_id=byoc.upload_checkpoint(
    model_name=checkpoint['model_name'],
    checkpoint_path=checkpoint['checkpoint_path'],
    description=checkpoint['description'],
    publisher=checkpoint['publisher'],
    param_count=checkpoint['param_count'],
    model_arch=checkpoint['model_arch'],
    seq_length=checkpoint['seq_length'],
    vocab_size=checkpoint['vocab_size'],
    app_id=checkpoint['app_id'],
    retries=3
)

2025-04-30 11:49:21,390 [INFO] Model with name 'Llama-3.1-8B-UltraMedical' not found
2025-04-30 11:49:21,556 [INFO] App with name 'eb0aaad1-694f-41b6-958a-b974737635c4' found with id eb0aaad1-694f-41b6-958a-b974737635c4
2025-04-30 11:49:21,556 [INFO] running snapi upload command:
 snapi import model create --model-name Llama-3.1-8B-UltraMedical --app eb0aaad1-694f-41b6-958a-b974737635c4 --source-type LOCAL --source-path /Users/francescar/Documents/ai-starter-kit/e2e_draft_model_training/data/models/models--TsinghuaC3I--Llama-3.1-8B-UltraMedical/snapshots/c982b9e34f7cf396f41627e7d851629f9e4e5792 --model-arch llama --parameter-count 8b --sequence-length 131072 --vocab-size 128256 -ni --publisher TsinghuaC3I --description Llama-3.1-8B-UltraMedical is an open-access large language model (LLM) specialized in biomedicine. Developed by the Tsinghua C3I Lab, this model aims to enhance medical examination access, literature comprehension, and clinical knowledge.
This could take a while
2025-04-

In [17]:
# Check the status of the uploaded checkpoint 
byoc.get_checkpoints_status(model_id)

2025-04-30 12:19:51,184 [INFO] model b5ca8c6a-e69e-434f-a72e-b75e0d19bda6 status: 
 {'model_id': 'b5ca8c6a-e69e-434f-a72e-b75e0d19bda6', 'status': '', 'progress': 0, 'stage': 'Copy', 'status_code': 200, 'headers': {'Date': 'Wed, 30 Apr 2025 11:19:51 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-headers': 'Accept, Content-Type, Content-Length, Accept-Encoding, Authorization, ResponseType, Access-Control-Allow-Origin', 'access-control-allow-methods': 'GET, POST, PATCH, DELETE', 'access-control-allow-origin': 'http://sn-studio-internal-1.cloud.snova.ai', 'content-security-policy': "default-src 'self'", 'permissions-policy': 'none', 'referrer-policy': 'no-referrer', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'x-content-type-options': 'nosniff', 'x-correlation-id': '41262a7d-bb14-4ad1-81f5-afe3aa43bc18', 'x-frame-options': 'DENY', 'x-envoy-upstream-service-time': '20', 'content-encoding': 

[{'model_id': 'b5ca8c6a-e69e-434f-a72e-b75e0d19bda6',
  'status': '',
  'progress': 0,
  'stage': 'Copy',
  'status_code': 200,
  'headers': {'Date': 'Wed, 30 Apr 2025 11:19:51 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-headers': 'Accept, Content-Type, Content-Length, Accept-Encoding, Authorization, ResponseType, Access-Control-Allow-Origin', 'access-control-allow-methods': 'GET, POST, PATCH, DELETE', 'access-control-allow-origin': 'http://sn-studio-internal-1.cloud.snova.ai', 'content-security-policy': "default-src 'self'", 'permissions-policy': 'none', 'referrer-policy': 'no-referrer', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'x-content-type-options': 'nosniff', 'x-correlation-id': '41262a7d-bb14-4ad1-81f5-afe3aa43bc18', 'x-frame-options': 'DENY', 'x-envoy-upstream-service-time': '20', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding'}}]